In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Set Up Driver

In [3]:
def login(cred, driver):
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')
        
    userfield = driver.find_element_by_name('email')
    passwordfield = driver.find_element_by_name('password')
    userfield.send_keys(user_password[0])
    passwordfield.send_keys(user_password[1])
    driver.find_element_by_name('submit').click()

In [4]:
kenpom_url = 'https://kenpom.com'

In [5]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [6]:
driver.get(kenpom_url)

In [7]:
cred = 'kenpom_cred.txt'
login(cred, driver)

In [8]:
selection_sundays = {2022: '2022-03-13',
                     2021: '2021-03-14',
                     2019: '2019-03-18',
                     2018: '2018-03-12',
                     2017: '2017-03-13',
                     2016: '2016-03-14',
                     2015: '2015-03-16',
                     2014: '2014-03-17',
                     2013: '2013-03-18',
                     2012: '2012-03-12',
                     2011: '2011-03-14',
                     2010: '2010-03-15',
                     2009: '2009-03-16',
                     2008: '2008-03-17'}

## Functions

In [9]:
def read_html(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [10]:
def get_season_data(soup, season):
    table = soup.find_all('table', {'id': 'ratings-table'})[0]
    data = [[td.text for td in tr.find_all('td')]
            for tbody in table.find_all('tbody')
            for tr in tbody.find_all('tr')]
    
    # Build dataframe
    df = pd.DataFrame(data).drop([18, 19, 20], axis=1)
    
    # Columns
    cols = ['SS_Rk', 'Team', 'Conf', 'SS_AdjEM']
    for col in ['AdjO', 'AdjD', 'AdjT']:
        cols.append(f'SS_{col}')
        cols.append(f'SS_{col}_Rk')
    cols = cols + ['Final_Rk', 'Final_AdjEM']
    for col in ['AdjO', 'AdjD', 'AdjT']:
        cols.append(f'Final_{col}')
        cols.append(f'Final_{col}_Rk')
    df.columns = cols
    
    # Final edit
    df['Season'] = season
    df['Team'] = df['Team'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))\
                           .apply(lambda x: x.strip())
    
    return df

In [11]:
def scrape_season_data(season, 
                       driver=driver, 
                       kenpom_url=kenpom_url,
                       selection_sundays=selection_sundays):
    ss_url = f'{kenpom_url}/archive.php?d={selection_sundays[season]}'
    driver.get(ss_url)
    soup = read_html(driver.page_source)
    return get_season_data(soup, season)

## Scrape

In [12]:
df = pd.concat([scrape_season_data(season) 
                for season in selection_sundays.keys()])\
       .reset_index().drop('index', axis=1)

In [13]:
driver.close()

## Save

In [34]:
df['Team'] = df['Team'].apply(lambda x: x.replace('Charleston', 'College of Charleston') 
                              if x == 'Charleston' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('IPFW', 'Purdue Fort Wayne') 
                              if x == 'IPFW' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Fort Wayne', 'Purdue Fort Wayne') 
                              if x == 'Fort Wayne' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Arkansas Little Rock', 'Little Rock') 
                              if x == 'Arkansas Little Rock' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('LIU Brooklyn', 'LIU') 
                              if x == 'LIU Brooklyn' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Louisiana Lafayette', 'Louisiana') 
                              if x == 'Louisiana Lafayette' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Texas Pan American', 'UT Rio Grande Valley') 
                              if x == 'Texas Pan American' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Detroit', 'Detroit Mercy') 
                              if x == 'Detroit' else x)
df['Team'] = df['Team'].apply(lambda x: x.replace('Utah Valley St.', 'Utah Valley') 
                              if x == 'Utah Valley St.' else x)

In [35]:
directory = 'MDataFiles_Stage2'
df_teams = pd.read_csv(f"{directory}/MTeams.csv")[['TeamID', 'TeamName']]
df_teams.head()

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [36]:
df_teams['TeamName'] = df_teams['TeamName']\
.apply(lambda x: x.replace('St', 'St.') 
       if 'State' not in x 
       and x != 'Stanford' and x != 'Stetson' and x != 'Stony Brook'
       else x)\
.apply(lambda x: x.replace('Loyola-Chicago', 'Loyola Chicago'))\
.apply(lambda x: x.replace('Bethune-Cookman', 'Bethune Cookman'))\
.apply(lambda x: x.replace('St. Louis', 'Saint Louis'))\
.apply(lambda x: x.replace("St. Mary's CA", "Saint Mary's"))\
.apply(lambda x: x.replace("St. Peter's", "Saint Peter's"))\
.apply(lambda x: x.replace("St. Joseph's PA", "Saint Joseph's"))\
.apply(lambda x: x.replace("Mt St. Mary's", "Mount St. Mary's"))\
.apply(lambda x: x.replace("Monmouth NJ", "Monmouth"))\
.apply(lambda x: x.replace('CS', 'Cal St.'))\
.apply(lambda x: x.replace('Cal St. Sacramento', 'Sacramento St.'))\
.apply(lambda x: x.replace('SUNY Albany', 'Albany'))\
.apply(lambda x: x.replace('Abilene Chr', 'Abilene Christian'))\
.apply(lambda x: x.replace('Central Conn', 'Central Connecticut'))\
.apply(lambda x: x.replace('Central Arkansas', 'Central Arkansas'))\
.apply(lambda x: x.replace('NC Central', 'North Carolina Central'))\
.apply(lambda x: x.replace('Boston Univ', 'Boston University'))\
.apply(lambda x: x.replace('Southern Univ', 'Southern'))\
.apply(lambda x: x.replace('American Univ', 'American'))\
.apply(lambda x: x.replace('Col Charleston', 'College of Charleston'))\
.apply(lambda x: x.replace('Charleston So', 'Charleston Southern'))\
.apply(lambda x: x.replace('Ark Pine Bluff', 'Arkansas Pine Bluff'))\
.apply(lambda x: x.replace('Ark Little Rock', 'Little Rock'))\
.apply(lambda x: x.replace('Cent Arkansas', 'Central Arkansas'))\
.apply(lambda x: x.replace('C Michigan', 'Central Michigan'))\
.apply(lambda x: x.replace('E Michigan', 'Eastern Michigan'))\
.apply(lambda x: x.replace('E Kentucky', 'Eastern Kentucky'))\
.apply(lambda x: x.replace('E Illinois', 'Eastern Illinois'))\
.apply(lambda x: x.replace('E Washington', 'Eastern Washington'))\
.apply(lambda x: x.replace('N Kentucky', 'Northern Kentucky'))\
.apply(lambda x: x.replace('N Colorado', 'Northern Colorado'))\
.apply(lambda x: x.replace('N Illinois', 'Northern Illinois'))\
.apply(lambda x: x.replace('N Dakota St.', 'North Dakota St.'))\
.apply(lambda x: x.replace('S Illinois', 'Southern Illinois'))\
.apply(lambda x: x.replace('S Carolina St.', 'South Carolina St.'))\
.apply(lambda x: x.replace('S Dakota St.', 'South Dakota St.'))\
.apply(lambda x: x.replace('SE Louisiana', 'Southeastern Louisiana'))\
.apply(lambda x: x.replace('SE Missouri St.', 'Southeast Missouri St.'))\
.apply(lambda x: x.replace('W Illinois', 'Western Illinois'))\
.apply(lambda x: x.replace('W Michigan', 'Western Michigan'))\
.apply(lambda x: x.replace('W Carolina', 'Western Carolina'))\
.apply(lambda x: x.replace('G Washington', 'George Washington'))\
.apply(lambda x: x.replace('Grambling', 'Grambling St.'))\
.apply(lambda x: x.replace('WKU', 'Western Kentucky'))\
.apply(lambda x: x.replace('NC A&T', 'North Carolina A&T'))\
.apply(lambda x: x.replace('NC State', 'N.C. State'))\
.apply(lambda x: x.replace('ULM', 'Louisiana Monroe'))\
.apply(lambda x: x.replace('PFW', 'Purdue Fort Wayne'))\
.apply(lambda x: x.replace('ETSU', 'East Tennessee St.'))\
.apply(lambda x: x.replace('MTSU', 'Middle Tennessee'))\
.apply(lambda x: x.replace('UTRGV', 'UT Rio Grande Valley'))\
.apply(lambda x: x.replace('UT San Antonio', 'UTSA'))\
.apply(lambda x: x.replace('IL Chicago', 'Illinois Chicago'))\
.apply(lambda x: x.replace('Coastal Car', 'Coastal Carolina'))\
.apply(lambda x: x.replace('FL Gulf Coast', 'Florida Gulf Coast'))\
.apply(lambda x: x.replace('Florida Intl', 'FIU'))\
.apply(lambda x: x.replace('Ga Southern', 'Georgia Southern'))\
.apply(lambda x: x.replace('Kennesaw', 'Kennesaw St.'))\
.apply(lambda x: x.replace('F Dickinson', 'Fairleigh Dickinson'))\
.apply(lambda x: x.replace('WI Green Bay', 'Green Bay'))\
.apply(lambda x: x.replace('WI Milwaukee', 'Milwaukee'))\
.apply(lambda x: x.replace('MA Lowell', 'UMass Lowell'))\
.apply(lambda x: x.replace('NE Omaha', 'Nebraska Omaha'))\
.apply(lambda x: x.replace('MS Valley St.', 'Mississippi Valley St.'))\
.apply(lambda x: x.replace('MD E Shore', 'Maryland Eastern Shore'))\
.apply(lambda x: x.replace('Missouri KC', 'UMKC'))\
.apply(lambda x: x.replace('Northwestern LA', 'Northwestern St.'))\
.apply(lambda x: x.replace('Citadel', 'The Citadel'))\
.apply(lambda x: x.replace('SF Austin', 'Stephen F. Austin'))\
.apply(lambda x: x.replace('TN Martin', 'Tennessee Martin'))\
.apply(lambda x: x.replace('FL Atlantic', 'Florida Atlantic'))\
.apply(lambda x: x.replace('Houston Bap', 'Houston Baptist'))\
.apply(lambda x: x.replace('Loy Marymount', 'Loyola Marymount'))\
.apply(lambda x: x.replace('SC Upstate', 'USC Upstate'))\
.apply(lambda x: x.replace('LIU Brooklyn', 'LIU'))\
.apply(lambda x: x.replace('TX Southern', 'Texas Southern'))\
.apply(lambda x: x.replace('SIUE', 'SIU Edwardsville'))\
.apply(lambda x: x.replace('TAM C. Christi', 'Texas A&M Corpus Chris'))\
.apply(lambda x: x.replace('Prairie View', 'Prairie View A&M'))\
.apply(lambda x: x.replace('Kent', 'Kent St.') if x == 'Kent' else x)\
.apply(lambda x: x.replace('Detroit', 'Detroit Mercy'))\
.apply(lambda x: x.replace('St. Thomas MN', 'St. Thomas'))\
.apply(lambda x: x.replace('W Salem St.', 'Winston Salem St.'))

In [38]:
output = df.merge(df_teams, left_on='Team', right_on='TeamName', how='left')\
           .drop('TeamName', axis=1)
output.head()

,SS_Rk,Team,Conf,SS_AdjEM,SS_AdjO,SS_AdjO_Rk,SS_AdjD,SS_AdjD_Rk,SS_AdjT,SS_AdjT_Rk,Final_Rk,Final_AdjEM,Final_AdjO,Final_AdjO_Rk,Final_AdjD,Final_AdjD_Rk,Final_AdjT,Final_AdjT_Rk,Season,TeamID
0,1,Gonzaga,WCC,+32.97,121.8,3,88.8,7,72.5,4,1,+32.90,121.8,1,88.9,7,72.5,4,2022,1211
1,2,Arizona,P12,+27.19,119.6,5,92.4,20,72.2,8,2,+27.24,119.6,5,92.4,20,72.2,8,2022,1112
2,3,Kentucky,SEC,+26.56,120.1,4,93.6,27,67.3,157,3,+26.65,120.2,4,93.5,25,67.4,153,2022,1246
3,4,Baylor,B12,+26.32,117.9,9,91.6,14,67.2,170,4,+26.52,117.9,8,91.4,14,67.2,169,2022,1124
4,5,Houston,Amer,+25.87,117.2,11,91.4,12,64.0,329,5,+26.51,117.3,10,90.8,11,63.9,333,2022,1222


In [39]:
# Check
output[output['TeamID'].isna()]

,SS_Rk,Team,Conf,SS_AdjEM,SS_AdjO,SS_AdjO_Rk,SS_AdjD,SS_AdjD_Rk,SS_AdjT,SS_AdjT_Rk,Final_Rk,Final_AdjEM,Final_AdjO,Final_AdjO_Rk,Final_AdjD,Final_AdjD_Rk,Final_AdjT,Final_AdjT_Rk,Season,TeamID


In [40]:
output.to_csv(f"{directory}/KenpomRating.csv", index=False)

In [41]:
output = pd.read_csv(f"{directory}/KenpomRating.csv")
output.head()

,SS_Rk,Team,Conf,SS_AdjEM,SS_AdjO,SS_AdjO_Rk,SS_AdjD,SS_AdjD_Rk,SS_AdjT,SS_AdjT_Rk,Final_Rk,Final_AdjEM,Final_AdjO,Final_AdjO_Rk,Final_AdjD,Final_AdjD_Rk,Final_AdjT,Final_AdjT_Rk,Season,TeamID
0,1,Gonzaga,WCC,32.97,121.8,3,88.8,7,72.5,4,1,32.90,121.8,1,88.9,7,72.5,4,2022,1211
1,2,Arizona,P12,27.19,119.6,5,92.4,20,72.2,8,2,27.24,119.6,5,92.4,20,72.2,8,2022,1112
2,3,Kentucky,SEC,26.56,120.1,4,93.6,27,67.3,157,3,26.65,120.2,4,93.5,25,67.4,153,2022,1246
3,4,Baylor,B12,26.32,117.9,9,91.6,14,67.2,170,4,26.52,117.9,8,91.4,14,67.2,169,2022,1124
4,5,Houston,Amer,25.87,117.2,11,91.4,12,64.0,329,5,26.51,117.3,10,90.8,11,63.9,333,2022,1222


In [42]:
ss_cols = ['SS_Rk', 'SS_AdjEM', 'SS_AdjO', 'SS_AdjD', 'SS_AdjT']
conf_strength = output.groupby(['Season', 'Conf'])[ss_cols]\
                      .agg(['mean', 'median']).reset_index()
conf_strength_cols = ['Season', 'Conf']
for col in ss_cols:
    conf_strength_cols.append(f'{col}_Conf_mean')
    conf_strength_cols.append(f'{col}_Conf_median')
conf_strength.columns = conf_strength_cols
conf_strength

,Season,Conf,SS_Rk_Conf_mean,SS_Rk_Conf_median,SS_AdjEM_Conf_mean,SS_AdjEM_Conf_median,SS_AdjO_Conf_mean,SS_AdjO_Conf_median,SS_AdjD_Conf_mean,SS_AdjD_Conf_median,SS_AdjT_Conf_mean,SS_AdjT_Conf_median
0,2008,A10,107.857143,81.5,7.073571,9.750,107.957143,107.90,100.907143,100.25,64.757143,63.30
1,2008,ACC,54.583333,55.5,14.745833,12.830,111.200000,109.75,96.450000,96.65,66.916667,67.30
2,2008,AE,230.222222,226.0,-7.005556,-6.150,99.822222,99.80,106.811111,106.90,63.988889,63.10
3,2008,ASun,250.500000,278.0,-9.479167,-11.260,97.825000,96.65,107.316667,107.00,66.816667,67.15
4,2008,B10,74.272727,48.0,12.163636,13.720,108.654545,109.10,96.500000,94.90,62.472727,62.70
...,...,...,...,...,...,...,...,...,...,...,...,...
449,2022,SWAC,292.916667,307.5,-12.109167,-12.570,95.025000,95.90,107.150000,106.70,67.766667,67.70
450,2022,Slnd,289.250000,293.5,-11.671250,-10.975,98.200000,97.60,109.862500,111.40,69.387500,69.65
451,2022,Sum,229.700000,221.5,-5.668000,-4.245,103.910000,102.40,109.560000,108.10,67.800000,67.30
452,2022,WAC,191.846154,140.0,-2.123077,2.650,100.476923,100.20,102.607692,99.30,67.415385,66.20


In [43]:
conf_strength[conf_strength['Season'] == 2022]\
.sort_values(['SS_AdjEM_Conf_median'], ascending=False)

,Season,Conf,SS_Rk_Conf_mean,SS_Rk_Conf_median,SS_AdjEM_Conf_mean,SS_AdjEM_Conf_median,SS_AdjO_Conf_mean,SS_AdjO_Conf_median,SS_AdjD_Conf_mean,SS_AdjD_Conf_median,SS_AdjT_Conf_mean,SS_AdjT_Conf_median
428,2022,B12,33.100000,34.5,17.959000,15.440,110.270000,109.55,92.320000,92.45,66.480000,66.50
427,2022,B10,57.142857,39.0,13.585714,14.905,111.285714,109.90,97.721429,97.85,66.571429,66.50
448,2022,SEC,60.714286,44.0,14.157857,14.420,109.714286,109.75,95.550000,94.70,67.878571,67.60
429,2022,BE,67.181818,53.0,12.305455,13.470,109.381818,109.80,97.072727,97.40,67.436364,67.30
441,2022,MWC,103.545455,69.0,7.996364,11.670,106.390909,107.10,98.381818,98.30,65.827273,65.40
423,2022,ACC,89.933333,78.0,9.394000,10.390,109.146667,110.00,99.733333,99.60,66.340000,67.00
444,2022,P12,88.250000,87.0,10.220833,9.190,108.000000,107.25,97.775000,97.05,67.008333,66.30
426,2022,Amer,110.000000,101.0,7.343636,7.030,105.618182,104.50,98.263636,97.90,66.990909,67.30
440,2022,MVC,140.100000,106.0,3.895000,6.250,105.120000,105.20,101.210000,101.55,65.990000,66.45
453,2022,WCC,131.700000,121.5,7.007000,5.685,106.520000,107.90,99.530000,101.45,68.130000,68.40


In [44]:
output.merge(conf_strength, on=['Season', 'Conf'], how='left')\
      .to_csv(f"{directory}/KenpomRating.csv", index=False)